In [1]:
collection_name = "chap_03"
connection_string = "http://localhost:6333"

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

# Define and load an embedding model, nomic it's optmized for italian
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(
    url=connection_string,
)

# Retrieve embedding
qdrant = QdrantVectorStore(
    client=client,
    embedding=embeddings_model,
    collection_name=collection_name,

)

In [4]:
# Get retrieved
retriever = qdrant.as_retriever()

In [10]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate([
    ("system", """
     You are a helpful assistant answer the question based only on the following context: {context}.
     Answer in Italian. Always add in the anser the bible chapter and verses.
     Useful informations:
     1 cubit = 45,72 cm
     """),
    ("user", "Question: {question}")
])

In [11]:
# Import model
llm = OllamaLLM(model='gemma3')

In [12]:
from langchain_core.runnables import chain

@chain
def qa(question):
    # Get relevant docs
    retrieved_docs = retriever.invoke(question)
    formatted = prompt.invoke({"context":retrieved_docs, "question":question})
    return llm.invoke(formatted)

In [13]:
question = "dimensioni dell'arca in centimetri"

In [14]:
qa.invoke("dimensioni dell'arca in centimetri")

'Le dimensioni dell’arca sono: lunga 300 cubiti, larga 50 cubiti e alta 30 cubiti.\n\nCalcoliamo le dimensioni in centimetri:\n\n*   Lungo: 300 cubiti * 45,72 cm/cubito = 13716 cm\n*   Largo: 50 cubiti * 45,72 cm/cubito = 2286 cm\n*   Alto: 30 cubiti * 45,72 cm/cubito = 1371,6 cm\n\nQuindi, le dimensioni dell’arca sono: 13716 cm di lunghezza, 2286 cm di larghezza e 1371,6 cm di altezza. (Genesi 6:14-16)'